In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),   # for ResNet50
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [8]:
train_dataset = datasets.ImageFolder("/kaggle/input/brain-tumor-mri-dataset/Training", transform=transform)
test_dataset  = datasets.ImageFolder("/kaggle/input/brain-tumor-mri-dataset/Testing", transform=transform)

train_loader = DataLoader(
    train_dataset,
    batch_size=32,
    shuffle=True,
    num_workers=4,      
    pin_memory=True
)

test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=False,
    num_workers=4,
    pin_memory=True
)
num_classes = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


# normal resnet 50

In [9]:
model = models.resnet50(pretrained=True)

# Freeze backbone (for first experiment)
for param in model.parameters():
    param.requires_grad = False

# Replace final layer
model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, num_classes)
)

model = model.to(device)


In [10]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.fc.parameters(), lr=1e-3)


In [11]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Loss: {running_loss/len(train_loader):.4f} "
          f"Accuracy: {100*correct/total:.2f}%")


Epoch [1/10] Loss: 0.6543 Accuracy: 75.73%
Epoch [2/10] Loss: 0.4564 Accuracy: 82.91%
Epoch [3/10] Loss: 0.4231 Accuracy: 84.18%
Epoch [4/10] Loss: 0.3786 Accuracy: 85.98%
Epoch [5/10] Loss: 0.3976 Accuracy: 84.91%
Epoch [6/10] Loss: 0.3804 Accuracy: 85.59%
Epoch [7/10] Loss: 0.3754 Accuracy: 85.80%
Epoch [8/10] Loss: 0.3887 Accuracy: 85.61%
Epoch [9/10] Loss: 0.3721 Accuracy: 86.12%
Epoch [10/10] Loss: 0.3684 Accuracy: 86.29%


In [15]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100*correct/total:.2f}%")


Test Accuracy: 77.75%


# normal Efficient net b4

In [16]:
transform_b4 = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])


In [17]:
model = models.efficientnet_b4(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.classifier[1].in_features, num_classes)
)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.classifier.parameters(), lr=1e-3)


In [18]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Loss: {running_loss/len(train_loader):.4f} "
          f"Accuracy: {100*correct/total:.2f}%")


Epoch [1/10] Loss: 0.8876 Accuracy: 72.54%
Epoch [2/10] Loss: 0.5910 Accuracy: 80.43%
Epoch [3/10] Loss: 0.5142 Accuracy: 82.45%
Epoch [4/10] Loss: 0.4755 Accuracy: 83.68%
Epoch [5/10] Loss: 0.4505 Accuracy: 83.89%
Epoch [6/10] Loss: 0.4293 Accuracy: 84.98%
Epoch [7/10] Loss: 0.4199 Accuracy: 84.27%
Epoch [8/10] Loss: 0.4194 Accuracy: 84.46%
Epoch [9/10] Loss: 0.4076 Accuracy: 85.09%
Epoch [10/10] Loss: 0.4068 Accuracy: 85.02%


In [19]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100*correct/total:.2f}%")


Test Accuracy: 81.19%


# Fine tune Res net 50

In [23]:
model = models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.fc.in_features, 4)
)
for param in model.layer4.parameters():
    param.requires_grad = True

for param in model.fc.parameters():
    param.requires_grad = True
    model = model.to(device)

In [24]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4   # 🔥 lower than before
)

In [25]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Loss: {running_loss/len(train_loader):.4f} "
          f"Accuracy: {100*correct/total:.2f}%")


Epoch [1/10] Loss: 0.2712 Accuracy: 90.25%
Epoch [2/10] Loss: 0.0592 Accuracy: 98.09%
Epoch [3/10] Loss: 0.0306 Accuracy: 98.98%
Epoch [4/10] Loss: 0.0170 Accuracy: 99.46%
Epoch [5/10] Loss: 0.0183 Accuracy: 99.50%
Epoch [6/10] Loss: 0.0174 Accuracy: 99.52%
Epoch [7/10] Loss: 0.0127 Accuracy: 99.61%
Epoch [8/10] Loss: 0.0094 Accuracy: 99.71%
Epoch [9/10] Loss: 0.0160 Accuracy: 99.54%
Epoch [10/10] Loss: 0.0104 Accuracy: 99.73%


In [26]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100*correct/total:.2f}%")


Test Accuracy: 94.75%


# Fine tune Efficient net b4


In [33]:
model = models.efficientnet_b4(pretrained=True)

In [34]:
for param in model.parameters():
    param.requires_grad = False


In [35]:
model.classifier = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(model.classifier[1].in_features, 4)
)


In [36]:
for param in model.features[-2:].parameters():
    param.requires_grad = True

for param in model.classifier.parameters():
    param.requires_grad = True
model = model.to(device)

In [37]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(
    filter(lambda p: p.requires_grad, model.parameters()),
    lr=1e-4
)


In [38]:
epochs = 10

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print(f"Epoch [{epoch+1}/{epochs}] "
          f"Loss: {running_loss/len(train_loader):.4f} "
          f"Accuracy: {100*correct/total:.2f}%")


Epoch [1/10] Loss: 1.0255 Accuracy: 63.82%
Epoch [2/10] Loss: 0.4944 Accuracy: 81.50%
Epoch [3/10] Loss: 0.3826 Accuracy: 85.55%
Epoch [4/10] Loss: 0.3297 Accuracy: 87.57%
Epoch [6/10] Loss: 0.2601 Accuracy: 90.27%
Epoch [7/10] Loss: 0.2339 Accuracy: 91.55%
Epoch [8/10] Loss: 0.2170 Accuracy: 91.77%
Epoch [9/10] Loss: 0.1986 Accuracy: 92.14%
Epoch [10/10] Loss: 0.1810 Accuracy: 93.34%


In [ ]:
model.eval()
correct = 0
total = 0

with torch.no_grad():
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)

        outputs = model(images)
        _, predicted = torch.max(outputs, 1)

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100*correct/total:.2f}%")


# With Scheduling Resnet 50 fine tuned

In [ ]:

transform_resnet = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

transform_effnet = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

In [ ]:
def train_resnet():

    train_dataset = datasets.ImageFolder("Training", transform=transform_resnet)
    val_dataset   = datasets.ImageFolder("Testing", transform=transform_resnet)

    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True)
    val_loader   = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=4, pin_memory=True)

    model = models.resnet50(pretrained=True)

    # Freeze everything
    for param in model.parameters():
        param.requires_grad = False

    # Replace classifier
    model.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(model.fc.in_features, 4)
    )

    # Unfreeze last block
    for param in model.layer4.parameters():
        param.requires_grad = True

    for param in model.fc.parameters():
        param.requires_grad = True

    model = model.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=1e-4
    )

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.1,
        patience=2,
        verbose=True
    )

    best_acc = 0
    epochs = 12

    for epoch in range(epochs):
        model.train()
        train_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total

        scheduler.step(val_loss)

        print(f"Epoch [{epoch+1}/{epochs}] "
              f"TrainLoss: {train_loss/len(train_loader):.4f} "
              f"ValLoss: {val_loss/len(val_loader):.4f} "
              f"ValAcc: {val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_resnet50.pth")

    print("Best Val Accuracy:", best_acc)


# efficient net with scheduler

In [ ]:
def train_efficientnet():

    train_dataset = datasets.ImageFolder("Training", transform=transform_effnet)
    val_dataset   = datasets.ImageFolder("Testing", transform=transform_effnet)

    train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, num_workers=4, pin_memory=True)
    val_loader   = DataLoader(val_dataset, batch_size=16, shuffle=False, num_workers=4, pin_memory=True)

    model = models.efficientnet_b4(pretrained=True)

    # Freeze all
    for param in model.parameters():
        param.requires_grad = False

    # Replace classifier
    model.classifier = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(model.classifier[1].in_features, 4)
    )

    # Unfreeze last 2 feature blocks
    for param in model.features[-2:].parameters():
        param.requires_grad = True

    for param in model.classifier.parameters():
        param.requires_grad = True

    model = model.to(device)

    criterion = nn.CrossEntropyLoss()

    optimizer = optim.Adam(
        filter(lambda p: p.requires_grad, model.parameters()),
        lr=1e-4
    )

    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        mode='min',
        factor=0.1,
        patience=2,
        verbose=True
    )

    best_acc = 0
    epochs = 12

    for epoch in range(epochs):
        model.train()
        train_loss = 0

        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0
        correct = 0
        total = 0

        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)

                val_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        val_acc = 100 * correct / total

        scheduler.step(val_loss)

        print(f"Epoch [{epoch+1}/{epochs}] "
              f"TrainLoss: {train_loss/len(train_loader):.4f} "
              f"ValLoss: {val_loss/len(val_loader):.4f} "
              f"ValAcc: {val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            torch.save(model.state_dict(), "best_efficientnet_b4.pth")

    print("Best Val Accuracy:", best_acc)


In [ ]:
train_resnet()

In [ ]:
train_efficientnet()